In [1]:

from pathlib import Path
import logging

from src.anki import read_file, make_deck, make_model, make_notes, make_package, read_package
from src.media import add_images, add_phonetics, add_sounds
from src.log_config import configure_logging

configure_logging(ignore_libs=["PIL", "phonemizer"])

INPUT_DIR = Path("input/")
OUTPUT_DIR = Path("output/")
TEMPLATE_DIR = Path("template/")
MEDIA_DIR = Path("media/")
IMG_DIR = MEDIA_DIR / "img"
SOUND_DIR = MEDIA_DIR / "sound"
for d in [INPUT_DIR, OUTPUT_DIR, TEMPLATE_DIR, MEDIA_DIR, IMG_DIR, SOUND_DIR]:
    d.mkdir(parents=True, exist_ok=True)

language = "fr"

top_deck_name = "Français"
file_name = "Les_nombres"
file_name = "L'alphabet"
file_name = "Cours_01"
file_name = "Cours_01_QA"
file_name = "Cours_02"
file_name = "Cours_03"
file_name = "Cours_03_QA"

deck_name = top_deck_name + "::" + file_name
file_type = ".md"
df_file = INPUT_DIR / (file_name + file_type)
apkg_file = OUTPUT_DIR / (file_name + ".apkg")
style = TEMPLATE_DIR / "style.css"
img_files, sound_files = [],[]


In [2]:
df = read_file(df_file)
df = add_phonetics(df, language)
df, sound_files = add_sounds(df, SOUND_DIR, language, force_replace=False)
df, img_files = add_images(df, IMG_DIR, force_replace=False)
media_files = sound_files + img_files
deck = make_deck(df, deck_name)
model = make_model(df, style)
make_notes(deck, model, df)
make_package(deck, media_files=media_files, apkg_file=apkg_file)
df.tail()

[23:38:48] [INFO] [media] - Added phonetics for 53 words.
[23:38:48] [INFO] [media] - Added 53 images for 53 vocab.
[23:38:48] [INFO] [media] - Sound for 'la bise / le bisou' saved to media/sound/sound_5cf87f264927ee26.mp3
[23:38:48] [INFO] [media] - Sound for 'travailler' saved to media/sound/sound_55c286af7ef60ef5.mp3
[23:38:49] [INFO] [media] - Sound for 'le travail' saved to media/sound/sound_095cdb2199169fc6.mp3
[23:38:49] [INFO] [media] - Sound for 'le taf' saved to media/sound/sound_0722bc6a2be50968.mp3
[23:38:49] [INFO] [media] - Sound for 'chanter' saved to media/sound/sound_94c8bcf55d4bc323.mp3
[23:38:49] [INFO] [media] - Sound for 'penser à quelque chose/quelqu'un' saved to media/sound/sound_2eed97044de97104.mp3
[23:38:49] [INFO] [media] - Sound for 'chercher quelque chose/quelqu'un' saved to media/sound/sound_41700c1c114cfcf9.mp3
[23:38:50] [INFO] [media] - Sound for 'rencontrer quelqu'un' saved to media/sound/sound_3e8e2a76c9a27783.mp3
[23:38:50] [INFO] [media] - Sound for

,Front,Back,Extra,More,Phonetics,Image,Sound,Reverse
48,toi et moi,du und ich,= betonte Personalpronomen<br><br>kommen IMMER...,,/twˌa e mwˈa/,"<img src=""img_6abded107290c5d8.png"">",[sound:sound_6abded107290c5d8.mp3],
49,Et toi?,Und du?,Mise en valeur (betontes Personalpronomen),,/e twˈa?/,"<img src=""img_2f0957c3b3ee4d63.png"">",[sound:sound_2f0957c3b3ee4d63.mp3],
50,Je veux avoir une bonne note.,Ich möchte eine eine gute Note bekommen.,"In deinem Satz ""Je veux avoir une bonne note"" ...",,/ʒə- vˈøz avwˈaʁ yn bˈɔn nˈɔt./,"<img src=""img_25eec1f8776f648c.png"">",[sound:sound_25eec1f8776f648c.mp3],
51,"Je voudrais un café, s'il vous plaît.","Ich hätte gern einen Kaffee, bitte.",,,"/ʒə- vudʁˈɛz œ̃ kafˈe, sil vu plˈɛ./","<img src=""img_6629edc27a8430f2.png"">",[sound:sound_6629edc27a8430f2.mp3],
52,"Je peux ouvrir la fenêtre, s'il vous plaît ?",Kann ich bitte das Fenster öffnen?,,,"/ʒə- pˈøz uvʁˈiʁ la- fənˈɛtʁ, sil vu plˈɛ ?/","<img src=""img_3c85c50a9311e3ac.png"">",[sound:sound_3c85c50a9311e3ac.mp3],


In [6]:
import os
from pathlib import Path
from phonemizer.backend import EspeakBackend
from phonemizer.backend.espeak.wrapper import EspeakWrapper
# Set up espeak functio
if os.uname().sysname == 'Darwin':
    EspeakWrapper.set_library(Path("/opt/local/bin/espeak")) # macports version

language = "fr-fr"
backend = EspeakBackend(language=language, with_stress=True, preserve_punctuation=True)
phonemize = lambda x: f"{backend.phonemize([str(x)], strip=True)[0]}"


In [9]:
words = """faire.
je fais.
tu fais.
il, elle fait.
nous faisons.
vous faites.
ils, elles font."""

phonemize(words)


'fˈɛʁ.\nʒə- fˈɛ.\nty fˈɛ.\nˈil, ɛl fˈɛ.\nnu fəzˈɔ̃.\nvu fˈɛt.\nˈil, ɛl fˈɔ̃.'